In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd 
pd.set_option('display.max_columns', None)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('../../features.csv')
df.drop(columns='Unnamed: 0', inplace=True)

In [3]:
df.head()

,loan_status_trim,state_income,annual_inc_joint,indiv_app,dti_joint,days_from_1st_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,home_ownership,inq_fi,inq_last_12m,installment,int_rate,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,for_debt_consol,total_acc,revol_bal,revol_util,delinq_amnt,delinq_2yrs,mths_since_last_delinq,mths_since_last_major_derog
0,0,2,55000.0,1,5.91,4505.0,10,679.0,675.0,3600.0,1,3.0,4.0,123.03,13.99,3600.0,7.0,0.0,0.0,1,13.0,2765.0,29.7,0.0,0.0,30.0,30.0
1,0,1,65000.0,1,16.06,5844.0,10,719.0,715.0,24700.0,1,0.0,6.0,820.28,11.99,24700.0,22.0,0.0,0.0,0,38.0,21470.0,19.2,0.0,1.0,6.0,0.0
2,0,0,71000.0,0,13.85,5600.0,10,699.0,695.0,20000.0,1,2.0,1.0,432.66,10.78,20000.0,6.0,0.0,0.0,0,18.0,7869.0,56.2,0.0,0.0,0.0,0.0
3,0,3,110000.0,1,17.06,2647.0,10,789.0,785.0,35000.0,1,0.0,0.0,829.90,14.85,35000.0,13.0,0.0,0.0,1,17.0,7802.0,11.6,0.0,0.0,0.0,0.0
4,0,2,104433.0,1,25.37,6392.0,3,699.0,695.0,10400.0,1,2.0,3.0,289.91,22.45,10400.0,12.0,0.0,0.0,0,35.0,21929.0,64.5,0.0,1.0,12.0,0.0


In [4]:
df['loan_status_trim'] = df['loan_status_trim'].apply(lambda x: True if x == 1 else False)
df['loan_status_trim'] = df['loan_status_trim'].apply(lambda x: 0 if x == True else 1)

In [5]:
df.loan_status_trim.value_counts()

1    1991308
0     269360
Name: loan_status_trim, dtype: int64

In [6]:
df.isnull().sum()

loan_status_trim               0
state_income                   0
annual_inc_joint               0
indiv_app                      0
dti_joint                      0
days_from_1st_cr_line          0
emp_length                     0
fico_range_high                0
fico_range_low                 0
funded_amnt                    0
home_ownership                 0
inq_fi                         0
inq_last_12m                   0
installment                    0
int_rate                       0
loan_amnt                      0
open_acc                       0
pub_rec                        0
pub_rec_bankruptcies           0
for_debt_consol                0
total_acc                      0
revol_bal                      0
revol_util                     0
delinq_amnt                    0
delinq_2yrs                    0
mths_since_last_delinq         0
mths_since_last_major_derog    0
dtype: int64

In [7]:
df.columns

Index(['loan_status_trim', 'state_income', 'annual_inc_joint', 'indiv_app',
       'dti_joint', 'days_from_1st_cr_line', 'emp_length', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'home_ownership', 'inq_fi',
       'inq_last_12m', 'installment', 'int_rate', 'loan_amnt', 'open_acc',
       'pub_rec', 'pub_rec_bankruptcies', 'for_debt_consol', 'total_acc',
       'revol_bal', 'revol_util', 'delinq_amnt', 'delinq_2yrs',
       'mths_since_last_delinq', 'mths_since_last_major_derog'],
      dtype='object')

In [8]:
 X = df[['state_income', 'annual_inc_joint', 'indiv_app',
       'dti_joint', 'days_from_1st_cr_line', 'emp_length', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'home_ownership', 'inq_fi',
       'inq_last_12m', 'installment', 'int_rate', 'loan_amnt', 'open_acc',
       'pub_rec', 'pub_rec_bankruptcies', 'for_debt_consol', 'total_acc',
       'revol_bal', 'revol_util', 'delinq_amnt', 'delinq_2yrs',
       'mths_since_last_delinq', 'mths_since_last_major_derog']]
    
y = df['loan_status_trim']
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27, shuffle=True)

In [ ]:
# Grid search cross validation
grid={'C':[1e-3, 1e-2, 1, 1e2, 1e3], 'l1_ratio':[0, 0.25, 0.5, 0.75, 1]}

logistic = LogisticRegression(solver='saga', multi_class='auto', penalty='elasticnet', class_weight='balanced', random_state=27, n_jobs=7)
logistic_cv = GridSearchCV(logistic, grid, cv=5)
logistic_cv.fit(X_train, y_train)

print('Tuned hyperparameters : (best parameters) ', logistic_cv.best_params_)
print('accuracy: ', logistic_cv.best_score_)
